In [65]:
import numpy as np
import pandas as pd
import chardet
import os
import csv

In [208]:
f='APCspend2013.csv'
os.rename('WELLCOME_APCspend2013_forThinkful.csv', f)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'WELLCOME_APCspend2013_forThinkful.csv' -> 'APCspend2013.csv'

In [94]:
# Random Function Online to Fix UTF Decoding Problem

path =  f
newpath = path+'edited'

with open(path, 'r', encoding='utf-8', errors='ignore') as infile, open(path + 'edited', 'w') as outfile:
     inputs = csv.reader(infile)
     output = csv.writer(outfile)

     for index, row in enumerate(inputs):
         # Create file with no header
         if index == 0:
             continue
         output.writerow(row)

UnicodeEncodeError: 'charmap' codec can't encode character '\u2212' in position 114: character maps to <undefined>

In [95]:
# Random Function #2 Online to fix UTF Decoding Problem

def find_encoding(fname):
    r_file = open(fname, 'rb').read()
    result = chardet.detect(r_file)
    charenc = result['encoding']
    return charenc
 
 
my_encoding = find_encoding(newpath)
df = pd.read_csv(newpath, encoding=my_encoding)

In [97]:
# Rename Columns

df.columns = ['PMID', 'Publisher','Journal','Article','Cost']

In [100]:
# Turn Cost into Numeric with Apply Function

df['Cost'] = df['Cost'].apply(pd.to_numeric, errors='coerce')

In [122]:
# Create New Notes Column

df["CleanNotes"]=None

In [141]:
# Make Notes of Changes to Rows with Cost of 99999, then change these to None

df.loc[df.Cost==999999, 'CleanNotes'] = "Cost erroneously entered as 999999.  Reset to None"
df.loc[df.Cost==999999, 'Cost'] = None

In [166]:
# Group By Journals and Aggregate with Count

df.groupby("Journal").count().sort_values("Article", ascending=False)["Article"].head()

Journal
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Human Molecular Genetics                           18
Proceedings of the National Academy of Sciences    17
Nature Communications                              17
Name: Article, dtype: int64

In [175]:
# Get Name of Journals of Interest for Stats

Journals=list(df.groupby("Journal").count().sort_values("Article", ascending=False)["Article"].head().index)

In [190]:
# Create Stat Tables

mymean=round(df.groupby("Journal").mean().loc[Journals,:],0)
mystd=round(df.groupby("Journal").std().loc[Journals,:],0)
mymedian=round(df.groupby("Journal").median().loc[Journals,:],0)
mycount=round(df.groupby("Journal").count().loc[Journals,:],0)["Article"]

In [205]:
# Concatenate Stats Table

Stats=pd.concat([mymean,mystd,mymedian,mycount],axis=1)

# Rename Columns

Stats.columns=["Mean","STD","Median","Article Count"]

In [209]:
Stats

,Mean,STD,Median,Article Count
Journal,,,,
Journal of Biological Chemistry,1430.0,426.0,1287.0,48
Nucleic Acids Research,1118.0,442.0,852.0,21
Human Molecular Genetics,2041.0,95.0,2040.0,18
Proceedings of the National Academy of Sciences,719.0,169.0,732.0,17
Nature Communications,3309.0,687.0,3642.0,17


In [221]:
Stats.loc[(1000<Stats["Mean"])&(3000>Stats["Mean"]),"Mean"]

Journal
Journal of Biological Chemistry    1430.0
Nucleic Acids Research             1118.0
Human Molecular Genetics           2041.0
Name: Mean, dtype: float64